In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA1_J/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2822,72.369670,0.256686,14.674241,0.677425,1.932593,0.042311,0.714639,1.465769,3.810637,4.056028
2823,72.372309,0.256559,14.673412,0.677306,1.932038,0.042305,0.714130,1.464871,3.810593,4.056477
2824,72.373871,0.256484,14.672921,0.677234,1.931710,0.042301,0.713829,1.464340,3.810567,4.056743
2825,72.374252,0.256465,14.672801,0.677217,1.931629,0.042300,0.713755,1.464210,3.810561,4.056808
2826,72.373405,0.256506,14.673067,0.677256,1.931808,0.042302,0.713919,1.464499,3.810575,4.056664


In [12]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2822,72.369670,0.256686,14.674241,0.677425,1.932593,0.042311,0.714639,1.465769,3.810637,4.056028
2823,72.372309,0.256559,14.673412,0.677306,1.932038,0.042305,0.714130,1.464871,3.810593,4.056477
2824,72.373871,0.256484,14.672921,0.677234,1.931710,0.042301,0.713829,1.464340,3.810567,4.056743
2825,72.374252,0.256465,14.672801,0.677217,1.931629,0.042300,0.713755,1.464210,3.810561,4.056808
2826,72.373405,0.256506,14.673067,0.677256,1.931808,0.042302,0.713919,1.464499,3.810575,4.056664
...,...,...,...,...,...,...,...,...,...,...
184622,70.805561,0.334702,15.096115,0.743794,2.251843,0.045496,1.047797,2.036020,3.826218,3.812454
184623,70.743929,0.337870,15.110124,0.746224,2.264013,0.045610,1.062168,2.059953,3.826440,3.803669
184624,70.668072,0.341777,15.127117,0.749195,2.278948,0.045750,1.079982,2.089551,3.826678,3.792931
184625,70.576138,0.346525,15.147347,0.752769,2.296982,0.045918,1.101755,2.125630,3.826912,3.780025


In [13]:
mineralogy1 = mineralogy.iloc[0:45000]
#mineralogy2 = mineralogy.iloc[45001:90000]

In [14]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA1_J/area1_J_1.txt", dataset_name="Vistelius (1995)")
#cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA1/area1_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [17]:
df_final = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1_J/area1_J_1_results_excel.txt")
#df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1/area1_2_results_excel.txt")

In [18]:
df_final

,Q,P,K
SAMPLE,,,
2822,29.268646,36.289389,27.191834
2823,29.272873,36.284544,27.194453
2824,29.275322,36.281763,27.195969
2825,29.275938,36.281043,27.196377
2826,29.274630,36.282556,27.195498
...,...,...,...
184622,26.641554,39.236911,25.765672
184623,26.532579,39.357297,25.713975
184624,26.397905,39.505963,25.650715


## QAPF classification

In [19]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [20]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [21]:
df_final.to_excel("../_CIPW/CIPW/AREA1_J/qapf.xlsx", index=True)

-------

In [22]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA1_J/QAPF_counts.xlsx")

-----

In [23]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area1_J.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [26]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control
QAPF_control.to_excel("../_CIPW/CIPW/AREA1_J/QAPF_control_points.xlsx", index=True)